In [23]:
import os, sys

# find repo root (looks for liars_poker/ or pyproject.toml)
def find_repo_root(start_dir: str) -> str:
    cur = os.path.abspath(start_dir)
    for _ in range(6):
        if os.path.isdir(os.path.join(cur, "liars_poker")) or os.path.exists(os.path.join(cur, "pyproject.toml")):
            return cur
        parent = os.path.dirname(cur)
        if parent == cur:
            break
        cur = parent
    return os.path.abspath(os.path.join(start_dir, "..", ".."))

NB_DIR = os.getcwd()
REPO_ROOT = find_repo_root(NB_DIR)
if REPO_ROOT not in sys.path:
    sys.path.insert(0, REPO_ROOT)

ARTIFACTS_ROOT = os.path.join(REPO_ROOT, "artifacts")
os.makedirs(ARTIFACTS_ROOT, exist_ok=True)

print("repo root   :", REPO_ROOT)
print("artifacts   :", ARTIFACTS_ROOT)


repo root   : /root/liars_poker
artifacts   : /root/liars_poker/artifacts


In [24]:
import random
from pprint import pprint

from liars_poker import (
    GameSpec, Env, InfoSet, Rules,
    best_response_mc, 
    Policy, TabularPolicy, CommitOnceMixture, RandomPolicy,
    eval_both_seats
)

from typing import List, Tuple

SEED = 42
random.seed(SEED)

# small game; P1 always starts by design
spec = GameSpec(ranks=6, suits=2, hand_size=2, claim_kinds=("RankHigh", "Pair"))
rules = Rules(spec)


In [25]:
def flatten_commit_once(policy: Policy) -> List[Tuple[Policy, float]]:
    if isinstance(policy, CommitOnceMixture):
        return list(zip(policy.policies, policy.weights))
    return [(policy, 1.0)]

def mix_policies(base_policy: Policy, br_policy: Policy, eta: float, rng: random.Random | None = None) -> CommitOnceMixture:
    base_components = flatten_commit_once(base_policy)
    br_components = flatten_commit_once(br_policy)

    combined_policies: List[Policy] = []
    combined_weights: List[float] = []

    for policy, weight in base_components:
        scaled = (1.0 - eta) * weight
        combined_policies.append(policy)
        combined_weights.append(scaled)

    for policy, weight in br_components:
        scaled = eta * weight
        combined_policies.append(policy)
        combined_weights.append(scaled)

    mixed_policy = CommitOnceMixture(combined_policies, combined_weights, rng=rng)
    mixed_policy.bind_rules(base_policy._rules)

    return mixed_policy

In [26]:
a0 = RandomPolicy()
a0.bind_rules(rules=rules)

# a0.store_efficiently('/root/liars_poker/artifacts/runs/run_temp')

In [27]:
all_averages = [a0]
all_brs = []

curr_av = a0

In [28]:

episodes = 10_000
last_exploitablity = 1

for i in range(100):
    print(i)
    eta = (1/(i+2))

    if last_exploitablity < 0.55:
        episodes = int(episodes*1.5)
        episodes = min(episodes, 200_000)
    elif last_exploitablity > 0.6:
        episodes = int(episodes*0.8)
        episodes = max(100, episodes)

    print(episodes)

    b_i = best_response_mc(spec=spec, opponent=curr_av, episodes=episodes, epsilon=0.1, min_visits_per_action=0, annotate='none', seed=i)
    all_brs.append(b_i)

    eval_results = eval_both_seats(spec, b_i, curr_av, episodes=10_000)
    print(eval_results)
    last_exploitablity = eval_results['A'] / eval_results['total']
    print(last_exploitablity)


    curr_av = mix_policies(curr_av, b_i, eta)
    all_averages.append(curr_av)




0
8000
{'A': 7042, 'B': 2958, 'total': 10000}
0.7042
1
6400
{'A': 6320, 'B': 3680, 'total': 10000}
0.632
2
5120
{'A': 6003, 'B': 3997, 'total': 10000}
0.6003
3
4096
{'A': 5518, 'B': 4482, 'total': 10000}
0.5518
4
4096
{'A': 5481, 'B': 4519, 'total': 10000}
0.5481
5
6144
{'A': 5722, 'B': 4278, 'total': 10000}
0.5722
6
6144
{'A': 5780, 'B': 4220, 'total': 10000}
0.578
7
6144
{'A': 5349, 'B': 4651, 'total': 10000}
0.5349
8
9216
{'A': 5879, 'B': 4121, 'total': 10000}
0.5879
9
9216
{'A': 5820, 'B': 4180, 'total': 10000}
0.582
10
9216
{'A': 5661, 'B': 4339, 'total': 10000}
0.5661
11
9216
{'A': 5650, 'B': 4350, 'total': 10000}
0.565
12
9216
{'A': 5530, 'B': 4470, 'total': 10000}
0.553
13
9216
{'A': 5659, 'B': 4341, 'total': 10000}
0.5659
14
9216
{'A': 5487, 'B': 4513, 'total': 10000}
0.5487
15
13824
{'A': 5924, 'B': 4076, 'total': 10000}
0.5924
16
13824
{'A': 5756, 'B': 4244, 'total': 10000}
0.5756
17
13824
{'A': 5695, 'B': 4305, 'total': 10000}
0.5695
18
13824
{'A': 5778, 'B': 4222, 'total':

KeyboardInterrupt: 

In [ ]:
iset = InfoSet(pid=1, hand=(2, ), history=(0, ))

print(curr_av.prob_dist_at_infoset(iset))




{-1: 0.16666666666666666, 1: 0.16666666666666666, 2: 0.16666666666666666, 3: 0.16666666666666666, 4: 0.16666666666666666, 5: 0.16666666666666666}


In [29]:
curr_av.store_efficiently('/root/liars_poker/artifacts/runs/run_temp_7')

In [30]:
curr_av.weights

[0.03333333333333335,
 0.03333333333333335,
 0.03333333333333335,
 0.03333333333333335,
 0.03333333333333335,
 0.03333333333333335,
 0.03333333333333335,
 0.03333333333333334,
 0.03333333333333334,
 0.033333333333333354,
 0.03333333333333335,
 0.033333333333333354,
 0.03333333333333335,
 0.03333333333333335,
 0.033333333333333326,
 0.03333333333333334,
 0.03333333333333334,
 0.03333333333333335,
 0.033333333333333354,
 0.03333333333333333,
 0.033333333333333354,
 0.033333333333333326,
 0.03333333333333335,
 0.03333333333333333,
 0.03333333333333333,
 0.03333333333333336,
 0.03333333333333333,
 0.03333333333333334,
 0.03333333333333333,
 0.03333333333333334]

In [ ]:
b_final = best_response_mc(spec, curr_av, episodes=1_000_000)

In [ ]:
eval_both_seats(spec, b_final, curr_av, episodes=10_000)

{'A': 7028, 'B': 2972, 'total': 10000}